## Analisando CVs de integrantes do Ministério das Mulheres, Família e Direitos Humanos

Esse código adapta uma ferramenta de análise de cúrriculos na área de recursos humanos. Assim, o modelo extrai textos de currículos de integrantes do MMFDH. A partir de termos encontrados no interior dos textos dos currículos, busco CVs que se enquadram num perfil específico [egressosos da Mackenzie, da Fundação Republicana Brasileira, formados em teologia e que utilizam a palavra "deus" em seus textos, entre outras informações], e que reflete um perfil de pessoas provenientes do campo religioso. As wordclouds nos auxilian na indentificação dessa tendencia, uma vez que a repetição de palavras e termos podem nos mostrar o perfil dos atores, que corresponde a uma análise qualitativa auxiliar.

### Intalar o pacote pdfplumber

In [ ]:
#pip install pdfplumber

### Bibliotecas

In [ ]:
import pdfplumber
import nltk

from wordcloud import WordCloud
import matplotlib.pyplot as plt

import requests
import csv
import sys

import pandas as pd
import numpy as np

from tqdm.auto import tqdm
import glob 
import os

### Diretório onde estão salvos os PDF´s dos CVs do MMFDH

In [ ]:
# é importante que insira o seu diretório aqui
e = r'C:\Users\Andre G\Documents\Pesquisa\Scripts Webscraping\Modelo (Currículos)\curriculos'

### Obter os endereços com os nomes dos arquivos

Juntar o diretório onde está o arquivo com o nome dos arquivos

In [ ]:
endereco = []
for i in os.listdir('./curriculos/'): # você poderá incluir o diretório, caso seja necessário.
    local = e + "\\" + i
    endereco.append(local)

### Quantidade de currículos

In [ ]:
len(endereco)

### Função para extrair e juntar o texto de várias páginas de um PDF

In [ ]:

def extract_pdf(pdf_path):
    all_text = ''
    with pdfplumber.open(pdf_path) as pdf:
        for pdf_page in pdf.pages:
            single_page_text = pdf_page.extract_text()
            all_text = all_text + '\n' + single_page_text
    return all_text

### Salva texto sujo dos currículos

### Salva texto limpo

### Função para salvar currículos e wordcloud: 

1) wordcloud´s de cada currículo;                             
2) aquivo csv com os textos dos currículos sem limpeza;                                     
3) aquivo csv com os textos dos currículos sem pontuação e sem stopwords.

In [ ]:
def wordcloud(cv, salvar=True):
    '''
    cv: diretório do arquivo PDF
    '''
    cv_limpo = []
    texto = []
    curriculos = []
    
    for arquivo in endereco:
        try:
            nome = arquivo.split('\\')[-1].replace('.pdf','').replace('.docx','')
            print(nome)
            tex = extract_pdf(arquivo)

            # Salva CSV texto sujo
            te = extract_pdf(arquivo).split()
            low = [palavra.lower() for palavra in te]
            tex = " ".join(s for s in low)
            texto.append([nome,tex])
            curriculos = pd.DataFrame(texto,columns=['Nome', 'Texto'] )
            curriculos.to_csv('texto_completo_curriculos.csv', sep=';', encoding='utf16')

            lista_de_palavras = nltk.tokenize.word_tokenize(tex)
            lista_de_palavras = [palavra.lower() for palavra in lista_de_palavras]

            stop_words = nltk.corpus.stopwords.words('portuguese')

            pontuação =['(',')',':','[',']',',','|',';',',','.','_']

            keywords = [palavra for palavra in lista_de_palavras if not palavra in stop_words and not palavra in pontuação]

            textocv = " ".join(s for s in keywords)

            #Salva CSV texto limpo
            cv_limpo.append([nome,textocv])
            cvlimpo = pd.DataFrame(cv_limpo,columns=['Nome', 'Texto_cv_limpo'] )
            cvlimpo.to_csv('texto_cv_limpo.csv', sep=';', encoding='utf16')

            wordcloud = WordCloud(background_color = '#0f54c9',
                                 max_font_size = 150,
                                 width = 1280,
                                 height = 720, 
                                 colormap = "Blues",
                                 max_words=50).generate(textocv)
            fig, ax = plt.subplots(figsize=(16,9))
            ax.imshow(wordcloud)
            ax.set_axis_off()
            plt.imshow(wordcloud)
            if salvar:
                wordcloud.to_file(nome + '_wordcloud.png')

            plt.show()
            
        except Exception:
            continue

### Quantidade de textos de currículos "sujos" e "limpos"

In [ ]:
#len(curriculos),len(cv_limpo)

### Chamando a função que salva os bancos e também salva as nuvens de palvras.

In [ ]:
wordcloud(endereco)

## Dando match em CVs do MMFDH

In [ ]:
import pandas as pd
vagas = pd.read_excel('dados_formação.xlsx', sheet_name = None)

In [ ]:
n_vagas = len(vagas.keys())
nome_vagas = list(vagas.keys())
n_vagas

In [ ]:
nome_vagas

In [ ]:
vagas = [vagas[nome_vagas[i]] for i in range(n_vagas)]

In [ ]:
# perfil alvo
vagas[4]

In [ ]:
# Definindo os termos da busca
palavras_chaves = list(vagas[4]['palavras-chave'])
palavras_chaves

In [ ]:
#criando a lista de dados biograficos
lista_de_vagas = vagas

## Função de entrada: CVs saida por vaga segundo score

In [ ]:
pontuacao = ['(', ')', ';', ':', '[', ']', ',']
stop_words = nltk.corpus.stopwords.words('portuguese')
cv = list(endereco)

def MatchCV(cv, vaga, limite = 5):
    '''
    cv: caminho de um arquivo PDF
    vaga: dataset de palavras-chave e pesos
    '''
    
    for arquivo in cv:
        nome = cv.split('\\')[-1].replace('.pdf','').replace('.docx','')
        tex = extract_pdf(cv)

        lista_de_palavras = nltk.tokenize.word_tokenize(tex)
        lista_de_palavras = [palavra.lower() for palavra in lista_de_palavras]

        keywords = [palavra for palavra in lista_de_palavras if not palavra in stop_words and not palavra in pontuacao]

        textocv = " ".join(s for s in keywords)

        pesos = list(vaga['pesos'])
        palavras_chaves = list(vaga['palavras-chave'])

        cont = [textocv.count(pc) for pc in palavras_chaves]  # conta quantas vezes cada termo da vaga aparece no texto do cv

        def aux(x, limite):
            return x if x <= limite else limite

        cont = [aux(i, limite) for i in cont]   # coloca o limite na contagem de palavras

        pmax = np.sum(np.array(pesos) * limite) 

        score = ((np.array(cont) * pesos).sum()/pmax).round(4)

        return score
       

### Lista de Listas: cada lista interna mostra o score de uma pessoa em relação ao perfil alvo do teste

In [ ]:
pessoas = [[MatchCV(cvs, vaga) for vaga in lista_de_vagas] for cvs in cv]
pessoas

In [ ]:
nomes = [arquivo.split('\\')[-1].replace('.pdf','').replace('.docx','') for arquivo in endereco]

In [ ]:
# Conjunto de dados comos matches das pessoas com as vagas
matchs = pd.DataFrame(pessoas, columns = nome_vagas, index = nomes)

In [ ]:
nome_vagas

In [ ]:
match = matchs.sort_values(by = 'curriculos_religiosos', ascending = False)
match.to_excel('matchs.xlsx', encoding="CP1252")
match

Tem formação superior ou maior?

In [ ]:
(match.formação.value_counts(normalize=True,ascending=True)*100).plot.barh()

É servidor de carreira?

In [ ]:
(match.servidor_carreira.value_counts(normalize=True,ascending=True)*100).plot.barh()

### Perfil alvo:

In [ ]:
match[match['curriculos_religiosos']>0.0001]

In [ ]:
# % de currículos com perfil religioso
len(match[match['curriculos_religiosos']>0.0001])/133*100